# Workflow

Define strings that we will attempt to cluster.

In [8]:
strings = ['donald trump', 'joe biden', 'mark esper', 'george floyds', 'christopher c . miller', 'jim mattis', 'nancy pelosi', 'trumps', 'trump', 'miller']


Compute matrix with mutual fuzzy ratios for strings

In [12]:
from fuzzup.gear import compute_fuzzy_matrix, compute_clusters, cluster_and_rank_strings
fuzzy_matrix = compute_fuzzy_matrix(strings)
fuzzy_matrix

strings,christopher c . miller,donald trump,george floyds,jim mattis,joe biden,mark esper,miller,nancy pelosi,trump,trumps
strings,,,,,,,,,,
christopher c . miller,100,18,17,25,32,31,43,18,22,21
donald trump,12,100,16,9,19,18,11,33,59,56
george floyds,11,8,100,17,36,26,11,32,11,21
jim mattis,12,18,17,100,32,30,25,18,13,25
joe biden,13,19,36,21,100,21,27,10,0,0
mark esper,31,27,17,30,21,100,38,36,27,25
miller,43,22,21,12,13,38,100,11,18,17
nancy pelosi,12,25,32,9,19,36,11,100,12,22
trump,15,59,11,13,0,27,18,12,100,91


Compute clusters from fuzzy ratios

In [13]:
compute_clusters(fuzzy_matrix)

[['donald trump', 'trump', 'trumps'],
 ['christopher c . miller', 'miller'],
 ['mark esper'],
 ['george floyds'],
 ['joe biden'],
 ['jim mattis'],
 ['nancy pelosi']]

Cluster strings and rank clusters in one go

In [ ]:
cluster_and_rank_strings(strings)